# Natural Language Processing

## 심화과제 1: BERT Fine-tunning with Transformers

> 본 과제는 NLP 심화구현 해보고자 하는 사람들을 위한 과제입니다.
>
> 정답이나 Reference 코드가 존재하지 않으므로 가능한 곳까지 도전해보세요!

### Introduction

* 본 과제는 imdb 영화 리뷰 데이터에 대해 pretrain 모델을 finetuning하는 과제입니다.
* 영화 리뷰가 주어졌을 때 긍정적인 리뷰인지 부정적인 리뷰인지 판별하는 모델을 만들어 봅시다.
* 이번 시간에은 산학계에서 실제로 많이 쓰이는 [Transformer](https://huggingface.co/docs/transformers/index) 라이브러리를 사용해보겠습니다. 해당 라이브러리를 직접 참고하면서 목표 정확도를 달성하는 것이 목표입니다.
* 모델, 초매개변수 (hyperparamter) 등등을 바꾸며 finetuning을 진행해서, 테스트 정확도 93% 이상을 넘겨보세요!
* 참고 1) https://huggingface.co/transformers/
* 참고 2) https://paperswithcode.com/sota/text-classification-on-imdb

### 0. 환경 셋팅 및 데이터 업로드

In [2]:
# !pip install transformers
# !pip install datasets

In [4]:
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

### 1. 데이터 전처리

In [5]:
import torch
from transformers import DistilBertTokenizerFast
from transformers import DistilBertConfig
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from pathlib import Path
from sklearn.model_selection import train_test_split

2023-04-06 19:03:35.354392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_5948/3331667923.py:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
  labels.append(0 if label_dir is "neg" else 1)


In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

토큰화기는 `BERT`에서 사용하는 토큰화기를 사용해보겠습니다.

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [9]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [10]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [16]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

20000 5000 25000


### 2. 모델 작성 및 학습
모델은 사전학습된 `BERT`를 증류 (Dilstilation) 과정을 통해 모델 크기를 줄인 `DistilBERT`를 사용해보겠습니다. 

In [17]:
config = DistilBertConfig.from_pretrained(
    'distilbert-base-uncased',
    vocab_size=30522, max_position_embeddings=512, sinusoidal_pos_embds=False,
    n_layers=6, n_heads=12, dim=768, hidden_dim=3072,
    dropout=0.1, attention_dropout=0.1, activation='gelu'
)

In [18]:
training_args = TrainingArguments(
    output_dir='./results',          # 출력 폴더
    num_train_epochs=1,              # 학습 에폭 수
    per_device_train_batch_size=16,  # GPU당 학습 배치 크기
    per_device_eval_batch_size=64,   # GPU당 평가 배치 크기기
    warmup_steps=500,                # 학습률 스케줄링을 위한 warm up 과정 스텝 수. 이동안은 학습률이 천천히 올라간다.
    weight_decay=0.01,               # 가중치 감쇠 (weight decay)
    logging_dir='./logs',            # 로그 기록을 위한 폴더
    logging_steps=100,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", config=config)

trainer = Trainer(
    model=model,                         # 학습할 모델
    args=training_args,                  # 학습 인자
    train_dataset=train_dataset,         # 학습 데이터 셋
    eval_dataset=val_dataset             # 평가 데이터 셋
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

Step,Training Loss
100,0.669600
200,0.348000
300,0.351000
400,0.303800
500,0.289100
600,0.287200
700,0.283500
800,0.316400
900,0.260000
1000,0.242100


TrainOutput(global_step=1250, training_loss=0.3134173599243164, metrics={'train_runtime': 460.832, 'train_samples_per_second': 43.4, 'train_steps_per_second': 2.712, 'total_flos': 2649347973120000.0, 'train_loss': 0.3134173599243164, 'epoch': 1.0})

### 3. 평가 코드

In [19]:
from datasets import load_metric
from torch.utils.data import DataLoader
from tqdm import tqdm

In [20]:
metric= load_metric("accuracy")
test_dataloader = DataLoader(test_dataset, batch_size=128)
model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to("cuda") for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

/tmp/ipykernel_5948/957560054.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric= load_metric("accuracy")


100%|███████████████████████████████████████████████████████████████████| 196/196 [03:02<00:00,  1.07it/s]


{'accuracy': 0.9258}

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
